In [2]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import sys, re
import plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from jitcache import Cache

survey_data = pd.read_csv('https://raw.githubusercontent.com/john-min/minsanity-medisprout/main/survey_data.csv?token=ADNMWRH56TOLZHCRA22YIDK72K3OI')

def parent_desktop(row):
    platform = row['platform']
    
    if row['is_mobile'] == 0:
        if bool(re.search('Mac', str(platform))) == True:
            return 'Mac OS'
        elif bool(re.search('Linux', str(platform))) == True:
            return 'Linux'
        elif bool(re.search('Windows', str(platform))) == True:
            return 'Windows'
        else:
            return 'NaN'
        
def mobile_devices(row):
    platform = row['platform']
    
    if row['is_mobile'] == 1:
        if bool(re.search('iPhone', str(platform))) == True:
            return 'iPhone'
        elif bool(re.search('iPad', str(platform))) == True:
            return 'iPad'
        elif bool(re.search('samsung', str(platform))) == True:
            return 'samsung'
        elif bool(re.search('vivo', str(platform))) == True:
            return 'vivo'
        elif bool(re.search('lenovo', str(platform))) == True:
            return 'lenovo'
        elif bool(re.search('LG', str(platform))) == True:
            return 'LG'
        elif bool(re.search('motorola', str(platform))) == True:
            return 'motorola'
        elif bool(re.search('Pixel', str(platform))) == True:
            return 'Google Pixel'
        elif bool(re.search('OnePlus', str(platform))) == True:
            return 'OnePlus'
        elif bool(re.search('Yulong', str(platform))) == True:
            return 'Yulong'
        elif bool(re.search('LGE', str(platform))) == True:
            return 'LGE'
        elif bool(re.search('TCL', str(platform))) == True:
            return 'TCL'
        elif bool(re.search('ZTE', str(platform))) == True:
            return 'ZTE'
        elif bool(re.search('Xiaomi', str(platform))) == True:
            return 'Xiaomi'
        elif bool(re.search('AlcatelOneTouch', str(platform))) == True:
            return 'AlcatelOneTouch'
        elif bool(re.search('HUAWEI', str(platform))) == True:
            return 'HUAWEI'
        elif bool(re.search('Amazon', str(platform))) == True:
            return 'Amazon'
        elif bool(re.search('ALCATEL', str(platform))) == True:
            return 'ALCATEL'
        elif bool(re.search('TINNO', str(platform))) == True:
            return 'TINNO'
        elif bool(re.search('BLU', str(platform))) == True:
            return 'BLU'
        elif bool(re.search('Alco', str(platform))) == True:
            return 'Alco'
        elif bool(re.search('Innovations', str(platform))) == True:
            return 'Innovations'
        elif bool(re.search('HMD', str(platform))) == True:
            return 'HMD/Nokia'
        elif bool(re.search('Sony', str(platform))) == True:
            return 'Sony'
        elif bool(re.search('HTC', str(platform))) == True:
            return 'HTC'
        elif bool(re.search('LENOVO', str(platform))) == True:
            return 'LENOVO'
        elif bool(re.search('asus', str(platform))) == True:
            return 'asus'
        elif bool(re.search('Alco', str(platform))) == True:
            return 'Alco'
        elif bool(re.search('LENOVO', str(platform))) == True:
            return 'LENOVO'
        else:
            return 'Other'

def classify_mobile_type(row):
    platform = row['platform']
    
    if row['is_mobile'] == 1:
        if bool(re.match('Apple iP\w*', str(platform))) == True:
            return 'iOS'
        else:
            return 'Android'
    else:
        return 'NA'
    
    return row

def parent_desktop(row):
    platform = row['platform']
    
    if row['is_mobile'] == 0:
        if bool(re.search('Mac', str(platform))) == True:
            return 'Mac OS'
        elif bool(re.search('Linux', str(platform))) == True:
            return 'Linux'
        elif bool(re.search('Windows', str(platform))) == True:
            return 'Windows'
        else:
            return 'NaN'

        
survey_data['mobile_type'] = survey_data.apply(lambda row: classify_mobile_type(row), axis=1)
survey_data['mobile_devices'] = survey_data.apply(lambda row: mobile_devices(row), axis=1)
survey_data['parent_desktop'] = survey_data.apply(lambda row: parent_desktop(row), axis=1)
survey_data['created_on'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f')
survey_data['created_on_date'] = survey_data['created_on'].dt.date
survey_data['months'] = pd.to_datetime(survey_data['created_on'], format='%Y-%m-%d %H:%M:%S.%f').dt.strftime('%Y-%m')


desktop_df = survey_data.groupby(['months', 'parent_desktop', 'source', 'is_satisfied'])['visit_id'].count().to_frame().reset_index()
mobile_df = survey_data.groupby(['months', 'mobile_devices', 'source', 'is_satisfied'])['visit_id'].count().to_frame().reset_index()
mom_df = survey_data.groupby(['months', 'is_satisfied', 'source', 'rating'])['visit_id'].count().to_frame().reset_index()

app = JupyterDash(__name__)
df1 = mom_df
df2 = mobile_df
df3 = desktop_df

layout1 = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='three columns'),

    html.Div([

        html.Br(),
        html.Label(['Rating'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='rating',
            options=[{'label':x, 'value':x} for x in df1.sort_values('rating')['rating'].unique()],
            value='1',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Satisfaction'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='is_satisfied',
            options=[{'label':x, 'value':x} for x in df1.sort_values('is_satisfied')['is_satisfied'].unique()],
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Months'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='months',
            options=[{'label':x, 'value':x} for x in df1.sort_values('months')['months'].unique()],
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        html.Div(id='source-display-value'),
        dcc.Link('Mobile Platforms|', href='/apps/mobile')
        
        


    ],className='one column'),

])

layout2 = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='three columns'),

    html.Div([

        html.Br(),
        html.Label(['Mobile Devices'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='mobile_devices',
            options=[{'label':x, 'value':x} for x in df2.sort_values('mobile_devices')['mobile_devices'].unique()],
            value='1',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Satisfaction'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='is_satisfied',
            options=[{'label':x, 'value':x} for x in df2.sort_values('is_satisfied')['is_satisfied'].unique()],
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Months'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='months',
            options=[{'label':x, 'value':x} for x in df2.sort_values('months')['months'].unique()],
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        html.Div(id='mobile-display-value'),
        dcc.Link('Desktop Platforms|', href='/apps/desktop')
        
        


    ],className='one column'),

])

layout3 = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='three columns'),

    html.Div([

        html.Br(),
        html.Label(['Desktop Type'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='parent_desktop',
            options=[{'label':x, 'value':x} for x in df3.sort_values('parent_desktop')['parent_desktop'].unique()],
            value='1',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Satisfaction'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='is_satisfied',
            options=[{'label':x, 'value':x} for x in df3.sort_values('is_satisfied')['is_satisfied'].unique()],
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        
        html.Br(),
        html.Label(['Months'],style={'font-weight': 'bold', "text-align": "center"}),
        dcc.Dropdown(id='months',
            options=[{'label':x, 'value':x} for x in df3.sort_values('months')['months'].unique()],
            value='',
            multi=False,
            disabled=False,
            clearable=True,
            searchable=True,
            placeholder='Choose Value...',
            className='form-dropdown',
            style={'width':"90%"},
            persistence='string',
            persistence_type='memory'),
        html.Div(id='desktop-display-value'),
        dcc.Link('source|', href='/apps/source')
        
        


    ],className='one column'),

])